<a href="https://colab.research.google.com/github/Riyan081/Python-CDAC/blob/main/whatsappchats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd

In [ ]:
f = open('_chats.txt','r',encoding='utf-8')

In [ ]:
data=f.read()

In [ ]:
print(data)

In [ ]:
import re

# Regex pattern to extract time and messages
pattern = r"\[([^]]+)\] (.+?(?=\n|\Z))"

matches = re.findall(pattern, data)

# Convert the matches into a list of lists
messages_list = [list(match) for match in matches]

messages_list


In [ ]:
import pandas as pd
df = pd.DataFrame(messages_list,columns=["date","user_message"])
df.head()

In [ ]:
df.shape

In [ ]:
df = df[df['date'].str.contains('AM|PM', case=False)]
df.shape

In [ ]:
#Converting dataframe to datetime datatype
df["date"] = pd.to_datetime(df["date"],format='%d/%m/%y, %I:%M:%S %p',errors="coerce")
df.shape

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Seperate users and messages
users = []
messages = []

for message in df["user_message"]:
    info = re.split('([\w\W]+?):\s',message)
    try:
        if info[1:]:
            users.append(info[1])
            messages.append(info[2])
        else:
            users.append('group_notification')
            messages.append(info[0])
    except Exception as e:
        raise("Error Occured")

df['user'] = users
df['messages'] = messages
df.drop(columns=['user_message'],inplace=True)

df.head(7)

In [ ]:
# Extracting the individul components of the date
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month_name()
df['year'] = df['date'].dt.year
df['hour'] = df['date'].dt.hour
df['min'] = df['date'].dt.minute
df['sec'] = df['date'].dt.second
df.head()

In [ ]:
df['user'].unique()

In [ ]:
#printing the total shape
print(df[df['user']=="Rachit"].shape)
#printing the number of rows alone
print(df[df['user']=="Rachit"].shape[0])

In [ ]:
words = []
for message in df['messages']:
    words.extend(message.split())


In [ ]:
df["messages"].value_counts()

In [ ]:
df[df["messages"] == "image omitted"].shape[0]

In [ ]:
df[df["messages"] == "\u200eThis message was deleted."].shape[0]

In [ ]:
df['messages'].str.contains('image omitted', case=False).sum()

In [ ]:
!pip install urlextract

In [ ]:
from urlextract import URLExtract

extractor = URLExtract()
links = []

for message in df["messages"]:
    links.extend(extractor.find_urls(message))

links

In [ ]:
message_counts = df["user"].value_counts().head(7)
message_counts

In [ ]:
import matplotlib.pyplot as plt


# Plot a bar graph
plt.bar(message_counts.index, message_counts.values)
plt.xlabel('User')
plt.ylabel('Number of Messages')
plt.title("Message Counts ")
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
# Add count annotations on top of each bar
for i, count in enumerate(message_counts.values):
    plt.text(i, count + 0.1, str(count), ha='center', va='bottom')

plt.show()

In [ ]:
new_df1 = round((df['user'].value_counts()/df.shape[0])*100,2).reset_index().rename(columns={'count':'percent'})
new_df1

In [ ]:
new_df2 = df["user"].value_counts().reset_index()
new_df2

In [ ]:
merged_df = pd.merge(new_df1, new_df2, on='user')
merged_df

In [ ]:
words = []

for message in df["messages"]:
    words.extend(message.split())

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
# remove all sticker/video/image/This "message was deleted.
ignore_text = ["\u200eimage omitted", "\u200eThis message was deleted." , "\u200evideo omitted" , "\u200esticker omitted"  ]
df_filtered = df[~df['messages'].isin(ignore_text)]
df_filtered

In [ ]:
#Remove Stopwords
f = open('stopwords.txt','r')
stop_words = f.read()

words = []

for message in df_filtered["messages"]:
    for word in message.lower().split():
        if word not in stop_words:
            words.append(word)

In [ ]:
from collections import Counter
pd.DataFrame(Counter(words).most_common(50))

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
timeline = df.groupby(['year','month']).count()['messages'].reset_index()

In [ ]:
timeline

In [ ]:
time = []

for i in range(timeline.shape[0]):
    time.append(timeline["month"][i] + "-" + str(timeline["year"][i]))

In [ ]:
timeline["time"] = time
timeline

In [ ]:
plt.plot(timeline["time"],timeline["messages"])
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
df["datee"] = df["date"].dt.date

In [ ]:
daily_timeline = df.groupby("datee").count()["messages"].reset_index()

In [ ]:
plt.plot(daily_timeline["datee"],daily_timeline["messages"])
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
df["day_name"] = df["date"].dt.day_name()
day_timeline = df["day_name"].value_counts().reset_index()
day_timeline

In [ ]:
plt.plot(day_timeline["day_name"],day_timeline["count"])
plt.xticks(rotation="vertical")
plt.show()

In [ ]:
df.head()

In [ ]:
df["hour"].unique()

In [ ]:
period = []

for hour in df[['day_name','hour']]['hour']:

    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour+1))
    else:
        period.append(str(hour) + "-" + str(hour+1))

df["period"] = period
df.head()

In [ ]:
import seaborn as sns
plt.figure(figsize=(18,5))
df['period'] = pd.Categorical(df['period'], categories=[f"{i:02d}-{(i + 1) % 24:02d}" for i in range(24)], ordered=True)
sns.heatmap(df.pivot_table(index="day_name",columns="period",values = "messages",aggfunc='count').fillna(0),cmap="crest")
plt.yticks(rotation="horizontal")
plt.show()